In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import os

# Семинар - Генерация текста

## 1. Подготовка
## 1.1. Выбор корпуса

Сегодня мы разберем принципы работы с рекурентными сетями с использованием фреймфорка Keras. 

Вашему Вниманию, представлено не сколько вариантов текста:

In [ ]:
directory = 'txt/'
files = os.listdir(directory)
files

Выберите понравившийся текст: 

In [ ]:
corpus = directory + files[1]
corpus

## 1.2. Предобработка текста

In [ ]:
with io.open(corpus, encoding='utf-8') as f:
    text = f.read().lower()
print('Длина корпуса:', len(text))

Посмотрим на часть текста:

In [ ]:
print(text[:100])

Сформируйте последовательности длинны `maxlen` со сдвигом `step`, следующим образом: 
- В переменную `sentences` включите последовательность
- `next_chars` список символов который является следующим для указанной последовательности

In [ ]:
maxlen = 70
step = 3
sentences = []
next_chars = []

# Ваш код здесь: 

In [ ]:
print('Колличество послеовательностей:', len(sentences))

Посмотрим на случайную пару: Последовательность/ответ

In [ ]:
check_ran = np.random.randint(0,len(sentences))
print('После строки:')
print(sentences[check_ran])
print('')
print('Ожидаем: ',next_chars[check_ran])

## 1.3. Векторизация
Далее, нам необходимо векторизвать каждую букву, для этого воспользуемся обычным Bag of chars:

In [ ]:
chars = sorted(list(set(text))) #Составив список всех символов
print('Всего символов:', len(chars))
# Составьте два словаря
# Первый: список букв:номера индексов
# Второй: номера индексов:список букв

# Ваш код здесь

Составьте обучающую выборку, где:  
- x - это закодированные последовательности, размерностью: (количество объектов, длинна последовательности, колличество уникальных букв), таким образом каждой букве в последовательности будет соответствовать вектор
- y - это закодированные ответы, размерностью: (количество объектов, колличество уникальных букв)

In [ ]:
print('Векторизация...', )
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.integer)
y = np.zeros((len(sentences), len(chars)), dtype=np.integer)


# Ваш код здесь

print('... Готово')

#### Проверим, размерность:

In [ ]:
x.shape, y.shape

# 2. Подготовка к генерации текста
Создадим функцию `generating_text`, которая будет генерировать объекты на основании моделей:  

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generating_text(diversity, model):
    print()
    print('...Генерация текста. Температура: ', diversity)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    sys.stdout.write(generated)
    for i in range(700):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0] #Вектор вероятностей, полученный с помощью модели
        next_index = sample(preds, diversity) # Выбираем индекс
        next_char = indices_char[next_index] #Выбираем символ из словаря 

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

> Temperature. We can also play with the temperature of the Softmax during sampling. Decreasing the temperature from 1 to some lower number (e.g. 0.5) makes the RNN more confident, but also more conservative in its samples. Conversely, higher temperatures will give more diversity but at cost of more mistakes (e.g. spelling mistakes, etc). In particular, setting temperature very near zero will give the most likely thing that Paul Graham might say:
>> “is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same”

> looks like we’ve reached an infinite loop about startups.

Источник: [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

# 3. Обучение моделей

In [ ]:
from keras.layers import SimpleRNN, GRU, LSTM 

## 3.1. VanilaRNN
Создадим простую RNN с одним слоем из 128 нейронов

In [ ]:
model = Sequential()
model.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

In [ ]:
model.fit(x, y, batch_size=128,
          epochs=10, validation_split=0.2)

#### Сгенерируем текст:

In [ ]:
generating_text(0.5, model)

Поэкспериментируйте с параметром `diversity` и посмотрите как изменяется генерируемый текст

## 3.2. Need more layers... 
Добавьте второй слой в нашу RNN, обучите и посмотрите на результат

In [ ]:
# Ваш код здесь: 

Проверьте какой текст получается при генерации для разного параметра `diversity`

In [ ]:
# Ваш код здесь: 

# 3.3 Больше экспериментов
Попробуйте различные варианты сетей: GRU и LSTM. Сравните результаты и выберите удачные примеры генерации текста. 